# Technical Factor - Advance Decline Trend Momentum (ADTM)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        IF(open <= m_lag(open, 1), 0, GREATEST(high - open, open - m_lag(open, 1))) AS DTM,
        IF(open >= m_lag(open, 1), 0, GREATEST(open - low,  open - m_lag(open, 1))) AS DBM,
        m_sum(DTM, 23) AS STM,
        m_sum(DBM, 23) AS SBM,
        IF(STM > SBM, (STM - SBM) / STM, (STM - SBM) / SBM) AS ADTM,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(ADTM > 0 AND m_lag(ADTM, 1) <= 0, 1, 0) AS TRBY1,
        IF(ADTM < 0 AND m_lag(ADTM, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(ADTM > 0 AND (ADTM - m_lag(ADTM, 1)) > 0, 1, 0) AS MTBY1,
        IF(ADTM < 0 AND (ADTM - m_lag(ADTM, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(ADTM < -0.5 AND (ADTM - m_lag(ADTM, 1)) > 0, 1, 0) AS RVBY1,
        IF(ADTM > 0.5 AND (ADTM - m_lag(ADTM, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(ADTM, 1) <= 0.3 AND ADTM > 0.3, 1, 0) AS BKBY1,
        IF(m_lag(ADTM, 1) >= -0.3 AND ADTM < -0.3, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        ADTM,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,ADTM,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-11,600519.SH,-0.084439,0,0,0,0,0,0,0,0
1,2025-02-12,600519.SH,0.136491,1,0,1,0,0,0,0,0
2,2025-02-13,600519.SH,0.390992,0,0,1,0,0,0,1,0
3,2025-02-14,600519.SH,0.531431,0,0,1,0,0,0,0,0
4,2025-02-17,600519.SH,0.582847,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
216,2025-12-25,600519.SH,-0.339270,0,0,0,0,0,0,0,0
217,2025-12-26,600519.SH,-0.180115,0,0,0,0,0,0,0,0
218,2025-12-29,600519.SH,-0.233931,0,0,0,1,0,0,0,0
219,2025-12-30,600519.SH,-0.323141,0,0,0,1,0,0,0,1
